In [1]:
import pandas as pd
import probablepeople as pp

In [2]:
def clean_company(raw_name):
    clean_name = ''
    short_name = ''
    for parsed_value, parsed_type in pp.parse(raw_name):
        if parsed_type == 'CorporationName':
            clean_name = clean_name + parsed_value + ' '
        if parsed_type == 'ShortForm':
            short_name = parsed_value
    if clean_name == '':
        return short_name
    else:
        return clean_name[:-1]

In [3]:
def extract_postcode(address):
    return ' '.join(address.split()[-2:])

In [4]:
mdf = pd.read_csv('R_P_Published_List_23_09_22_Amended.csv')  
mdf['COMPANY_CLEAN'] = mdf.apply(lambda row: clean_company(row['COMPANY']), axis=1)
mdf = mdf[mdf['COMPANY_CLEAN']!='']

In [5]:
mdf['POSTCODE'] = mdf.apply(lambda row: extract_postcode(row['ADDRESS & CONTACT DETAILS']), axis=1)
mdf['LOCATION'] = mdf['LOCATION'].str.upper()
mdf['COMPANY_CLEAN'] = mdf['COMPANY_CLEAN'].str.upper()

In [6]:
cdf = pd.read_csv('BasicCompanyData-2022-10-01-part1_7.csv')
cdf = cdf.rename(columns={"RegAddress.PostCode": "POSTCODE"})
cdf['COMPANY_CLEAN']=cdf['CompanyName']

C:\Users\mshea\opencv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (41,42,43,44,45,46,47,48,49,50,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
import recordlinkage

In [8]:
indexer = recordlinkage.Index()
indexer.block("POSTCODE")
candidate_links = indexer.index(cdf, mdf)
len(candidate_links)

23613

In [9]:
compare_cl = recordlinkage.Compare()
compare_cl.string("COMPANY_CLEAN", "COMPANY_CLEAN", method='jarowinkler',threshold=0.9)
compare_cl.string("RegAddress.PostTown", "LOCATION", method='jarowinkler',threshold=0.9)
features = compare_cl.compute(candidate_links, cdf, mdf)

In [10]:
features[(features[1]==1) & (features[0]==1)]

,,0,1
215972,4,1.0,1.0
575408,7,1.0,1.0
181344,2,1.0,1.0
181345,2,1.0,1.0
181346,2,1.0,1.0
183753,3,1.0,1.0
183754,3,1.0,1.0
183755,3,1.0,1.0
596106,8,1.0,1.0
257746,5,1.0,1.0
